In [ ]:
PLAYER_NAME = ""                    # Player of interest
ENGINE_NAME = "stockfish"           # Engine for single engine analysis
ENGINES = [ENGINE_NAME]             # Engines for which to retrieve game analyses
QUANTILE = 0.05                     # Best and worst Q% of games
R_MIN = 800                         # Floor for the worst performances
R_MAX = 2300                        # Ceiling for the best performances
EPSILON = 0.01                      # How close can get worst games get to R_min + epsilon*(R_max - R_min)

In [ ]:
%run database.ipynb
%run analysis.ipynb
%run export.ipynb

1. Load the game IDs for all games involving the player of interest
2. Calculate the summary statsitics, that include ACPL
3. Build a data frame containing the ACPL values for the player of interest:

| index | game_id | engine    | depth | acpl       |
| ----- | ------- | --------- | ----- | ---------- |
| 0     | 1       | stockfish | 0     | 91.680000  |
| 1     | 2       | stockfish | 0     | 112.692308 |
| 2     | 3       | stockfish | 0     | 22.057692  |
| 3     | 4       | stockfish | 0     | 79.285714  |
| 4     | 5       | stockfish | 0     | 67.312500  |

In [ ]:
import pandas as pd

# Load the games the player of interest has participated in
connection = connect()
games_df = load_player_game_ids(connection, PLAYER_NAME)

# Construct the dataframe to hold analysis results for the player of interest
player_acpl_df = pd.DataFrame(columns=[
        "game_id",
        "engine",
        "depth",
        "acpl"])

# Iterate over the games
for row in games_df.itertuples(index=False):
    # Extract the game ID and player colour for this game
    game_id = row.id
    player = row.Colour

    # Iterate over the engines of interest
    for engine in ENGINES:
        # Load the analysis and generate a summary
        analysis_df = load_analysis(connection, game_id, engine)
        summary_df = calculate_summary_statistics(analysis_df)

        # # Extract the summary data for the player
        player_summary_df = summary_df[summary_df["player"] == player]
        player_acpl_df.loc[len(player_acpl_df)] = [
            player_summary_df.iloc[0]["game_id"],
            player_summary_df.iloc[0]["engine"],
            player_summary_df.iloc[0]["depth"],
            player_summary_df.iloc[0]["acpl"]
        ]

# Preview the data
display(player_acpl_df.head())

Extract the ACPL values for the single engine of interest, determine the best and worst games and calculate K

In [ ]:
import numpy as np

# Extract the ACPL values for the engine of interest
df = player_acpl_df[player_acpl_df["engine"] == ENGINE_NAME].copy()

# Get the best and worst Q% of games
ACPL_best = df["acpl"].quantile(QUANTILE)
ACPL_worst = df["acpl"].quantile(1.00 - QUANTILE)

# K defines how quickly ELO drops as ACPL increases
K = np.log(1/EPSILON) / (ACPL_worst - ACPL_best)

print(f'ACPL(best) is {ACPL_best}')
print(f"K for games played by {PLAYER_NAME} and anlysed using {ENGINE_NAME} is {K}")

Having calculated "k" and determined "ACPL_best", the function personal_elo_engine() provides a way of estimating ELO for a game with ACPL equal to "acpl"

In [ ]:
def personal_elo_single_engine(acpl, ACPL_best=ACPL_best):
    return R_MIN + (R_MAX - R_MIN) * np.exp(-K * (acpl - ACPL_best))

df["elo_personal_single"] = personal_elo_single_engine(df["acpl"])
display(df)

A smoothed density plot shows the structure of ACPL values more clearly than a histogram and helping to motivate the exponential decay model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

# ACPL values
acpl = df["acpl"].values

# KDE
kde = gaussian_kde(acpl)

x = np.linspace(acpl.min(), acpl.max(), 400)
y = kde(x)

plt.figure(figsize=(8, 5))
plt.plot(x, y, linewidth=2)
plt.xlabel("ACPL")
plt.ylabel("Density")
plt.title(f"{ENGINE_NAME} ACPL Distribution")
plt.show()

# 

In [ ]:
import matplotlib.pyplot as plt

acpl_range = np.linspace(df["acpl"].min(), df["acpl"].max(), 200)
elo_curve = personal_elo_single_engine(acpl_range)

plt.figure(figsize=(8, 5))
plt.scatter(df["acpl"], df["elo_personal_single"], alpha=0.3, label="Games")
plt.plot(acpl_range, elo_curve, linewidth=2, label="Fitted curve")
plt.xlabel(f"{ENGINE_NAME} ACPL")
plt.ylabel("Personal Elo estimate")
plt.title(f"Personal ACPL → Elo model ({ENGINE_NAME} only)")
plt.legend()
plt.show()